# Initiailize VW executor

In [ ]:
from VwPipeline import Loggers, Handlers
from VwPipeline.VwCache import VwCache
from VwPipeline.Vw import Vw
from VwPipeline.VwOpts import dimension, product
import pandas as pd

#your vw path
vw_path = r'/bin/vw'

#path to local folder to keep intermediate results
cache = VwCache(r'_cache')

vw = Vw(vw_path,
    cache,
    handlers=[Handlers.WidgetHandler()],
#    loggers=[Loggers.ConsoleLogger('DEBUG')]
    )

# Get the data

In [ ]:
inputs = [
    r'cb-1.json',
    r'cb-2.json'
    ]
inputs

# Define options grid and train

In [ ]:
opts = pd.DataFrame(product(
    dimension('#base', ['--ccb_explore_adf -P 10000 --preserve_performance_counters --save_resume --dsjson']),
    dimension('--cb_type', ['ips', 'mtr']),
))
opts

In [ ]:
result = vw.train(inputs, opts)
result

# Explore metrics


In [ ]:
result.iloc[0]['!Metrics'].head()

In [ ]:
import matplotlib.pyplot as plt
[(-r['!Metrics']['loss_per_example']).plot(label=f"{r['!Job'].name}", figsize=(10,6)) for i, r in result[result['!Loss'].notnull()].iterrows()]
plt.legend()

# Choose the best and try other options on top

In [ ]:
best1 = result.sort_values(by='!Loss').head(1)
best1

In [ ]:
opts = pd.DataFrame(product(best1, 
    dimension('-l', ['0.1', '0.2'])))
opts

In [ ]:
result = vw.train(inputs, opts)
result